In [0]:
import numpy as np
data = np.random.rand(20000)
rdd = sc.parallelize(data)
print(rdd.getNumPartitions())


8

In [0]:
#Cela veut dire qu'on peut lancer 8 taches sur 8 noeud en paralléles (une tache (task) par partition)

In [0]:
stop_words = ['the', 'a', 'of']
rdd2 = sc.textFile('FileStore/tables/text_gradient.txt')
rdd_filtred= rdd2.map(lambda x: x.replace(' the ', ' ')).map(lambda x: x.replace(' a ', ' ')).map(lambda x: x.replace(' of ', ' '))
rdd_filtred.take(5)


Out[4]: ['Gradient descent is first-order iterative optimization algorithm for ',
 'finding local minimum differentiable function.',
 'The idea is to take repeated steps in opposite direction of',
 'the gradient (or approximate gradient) function at current point, ',
 'because this is direction steepest descent. ']

In [0]:
import numpy as np
import pandas as pd


def random_indexes(arr, rate = 0.1):
    index_of_arr = np.arange(len(arr))
    np.random.shuffle(index_of_arr)
    return index_of_arr[0:int(len(arr)*rate)]

def random_nan(arr, rate = 0.1):
    index_to_nan = random_indexes(arr, rate)
    if arr.dtype == np.int:
        arr = arr.astype('float')
    arr[index_to_nan] = np.nan
    return arr


def generate_random_string(vocab, string_length):

    return " ".join(list(np.random.choice(vocab, string_length)))


def generate_fake_email(n):
    """
    :param n: int number of email to generate
    :return: numpy array of emails
    """
    import faker
    fake = faker.Faker()
    return np.array([fake.email() for i in range(n)])


def format_column_name(col):

    return col.replace(' ', '_')


#mettre if pour is_nan => null (lowercase "nan")
def format_value(v):
    if isinstance(v, (str)):
        v = v.lower()
        if v == "nan":
            value = v.replace("nan", "NULL")
            return str(value)

        value = v.replace("'", "\\'")

        return "'" + str(value) + "'"
    if isinstance(v, np.float) and v is np.nan:
        return 'NULL'
    else:
        return str(v)

def create_table_schema(table_name, df):

    engine = create_engine('mysql+pymysql://', echo=False)

    sql = pd.io.sql.get_schema(df, table_name, con=engine).strip()
    return sql + ";"

def create_table_schemas(tables_names, dfs):

    sql = ""
    for table_name, df in zip(tables_names, dfs):
        sql += create_table_schema(table_name, df) + "\n"

    return sql

def df_to_sql_table(table_name, df):

    schema = create_table_schema(table_name, df)
    inserts = df_to_sql_insert(table_name, df)
    return "{}\n{}".format(schema, inserts)

def df_to_sql_insert(table_name, df):

    inserts = []
    for row in df.to_dict(orient='records'):
        insert = generate_insert_statement(table_name, row)
        inserts.append(insert)

    inserts_str = '\n'.join(inserts)

    return inserts_str + '\n'


def generate_insert_statement(table_name, values_as_dict):

    d = values_as_dict
    columns = ", ".join(list(d.keys()))
    values = ", ".join([format_value(v) for v in d.values()])

    s = "INSERT INTO {} ({}) VALUES ({});".format(table_name, columns, values)
    return s


def dfs_to_sql(tables_names, dfs, file_path=None):

    schemas = create_table_schemas(tables_names, dfs)

    inserts = ""
    for table_name, df in zip(tables_names, dfs):
        inserts += df_to_sql_insert(table_name, df)

    sql = "{}\n{}".format(schemas, inserts)

    if not file_path is None:
        with open(file_path, 'w') as f:
            f.write(sql)

    return sql



def df_to_sql_file(table_name, df, filepath):

    statements = df_to_sql_table(table_name, df)
    with open(filepath, 'w') as f:
        f.write(statements)


def generate_df(requirements, nrows):

    df = pd.DataFrame(index=np.arange(nrows))
    for requirement in requirements:

        colname = requirement['name']
        generator = requirement['generator']
        df[colname] = generator(nrows, df)

    return df




def sigmoid(x):
    return 1 / (1 + np.exp(-x))

In [0]:


import numpy as np

def generate_price(df, nrows):

    price = 45000 * np.log(1 + df['size'])
    price += 20000 * (df['nb_rooms'] - 1)

    price += 20000 * (df['orientation'] == 'Sud').astype('int')
    price -= 20000 * (df['orientation'] == 'Nord').astype('int')
    price += 1000 * (df['orientation'] == 'Ouest').astype('int')
    price -= 3000 * (df['orientation'] == 'Est').astype('int')

    price += 50000 * (df['garden'] == 1).astype('int')

    return price

def houses(name):

    requirements = [

        {
            'name': 'size',
            'generator': lambda nrows, df: np.abs(150 + 50 * np.random.randn(nrows))
        },
        {
            'name': 'nb_rooms',
            'generator': lambda nrows, df: np.random.randint(1, 4, nrows)
        },
        {
            'name': 'garden',
            'generator': lambda nrows, df: np.random.randint(0, 2, nrows)
        },
        {
            'name': 'orientation',
            'generator': lambda nrows, df: np.random.choice(['Sud', 'Est', 'Ouest', 'Nord'], nrows)
        },
        {
            'name': 'price',
            'generator': lambda nrows, df: generate_price(df, nrows)
        },
    ]

    df = generate_df(requirements, 100000000)


    df.to_csv(name, index=False)


def house_size_is_sold_in_six_month():
    requirements = [
        {
            'name': 'size',
            'generator': lambda nrows, df: 150 + 50 * np.random.randn(nrows),
        },
        {
            'name': 'sold_in_six_month',
            'is_target': True,
            'generator': lambda nrows, df: df['size'] > 150
        }
    ]

    df = generate_df(requirements, nrows=800000)

    df.to_csv('house_classification.csv')

houses('houses.csv')

In [0]:
df = pd.read_csv('houses.csv')
sparkDf=spark.createDataFrame(df)
sparkDf.printSchema()

root
-- size: double (nullable = true)
-- nb_rooms: long (nullable = true)
-- garden: long (nullable = true)
-- orientation: string (nullable = true)
-- price: double (nullable = true)

In [0]:
from pyspark.serializers import PickleSerializer, AutoBatchedSerializer
def _to_java_object_rdd(rdd):  
   
    rdd = rdd._reserialize(AutoBatchedSerializer(PickleSerializer()))
    return rdd.ctx._jvm.org.apache.spark.mllib.api.python.SerDe.pythonToJava(rdd._jrdd, True)

JavaObj = _to_java_object_rdd(sparkDf.rdd)

nbytes = sc._jvm.org.apache.spark.util.SizeEstimator.estimate(JavaObj)
print(nbytes)

4224424344

In [0]:
df_filtred = sparkDf.filter(sparkDf.nb_rooms > 2)
df_filtred.show(10)

+------------------+--------+------+-----------+------------------+
 size|nb_rooms|garden|orientation| price|
+------------------+--------+------+-----------+------------------+
137.30027526131454| 3| 1| Ouest| 312824.225304512|
148.59258955996802| 3| 1| Est|312356.19881789363|
192.67056190501847| 3| 1| Est|323977.13614897546|
118.48163264872223| 3| 0| Nord|235242.31960432642|
180.16142351158868| 3| 1| Est| 320972.4814387185|
167.61000386988792| 3| 1| Sud| 340741.477043372|
 155.4553500321651| 3| 1| Nord| 297374.6799684082|
131.83054115223706| 3| 1| Est|307008.33851207944|
200.06973888120532| 3| 1| Sud| 348664.3313370723|
 174.8451659379434| 3| 1| Sud|342632.17409832845|
+------------------+--------+------+-----------+------------------+
only showing top 10 rows

In [0]:
from pyspark.sql import functions as F
prix_et_surface_moyenne = sparkDf.agg(F.avg('price'),F.avg('size'))
print("le prix moyen du dataset original : ",prix_et_surface_moyenne.show())


+-----------------+-----------------+
 avg(price)| avg(size)|
+-----------------+-----------------+
267157.8635628571|150.0386842946822|
+-----------------+-----------------+

le prix moyen du dataset original : None

In [0]:
prix_et_surface_moyenne_filtred = df_filtred.agg(F.avg('price'),F.avg('size'))
prix_et_surface_moyenne_filtred.show()


+------------------+------------------+
 avg(price)| avg(size)|
+------------------+------------------+
287164.48468725494|150.05044084923279|
+------------------+------------------+

In [0]:
from pyspark.ml.feature import StringIndexer, OneHotEncoder
from pyspark.sql.functions import stddev, mean, col
df_stats = sparkDf.select(
    mean(col('size')).alias('mean'),
    stddev(col('size')).alias('std')
).collect()

mean = df_stats[0]['mean']
std = df_stats[0]['std']
sparkDf_normalized = sparkDf.withColumn('size', (F.col('size') - mean)/std)
sparkDf_normalized.show(5)
sparkDf.show(5)


+--------------------+--------+------+-----------+------------------+
 size|nb_rooms|garden|orientation| price|
+--------------------+--------+------+-----------+------------------+
-0.24755175254238312| 1| 1| Ouest| 272950.5282361945|
 -0.6210217492781106| 2| 0| Sud|255458.71474544148|
 0.06756213200245424| 3| 0| Ouest|267782.27342603524|
 -0.8436144711369394| 3| 0| Ouest|252091.25689685327|
-0.13101177840371883| 3| 0| Nord| 243798.4680452121|
+--------------------+--------+------+-----------+------------------+
only showing top 5 rows

+------------------+--------+------+-----------+------------------+
 size|nb_rooms|garden|orientation| price|
+------------------+--------+------+-----------+------------------+
137.68899230099348| 1| 1| Ouest| 272950.5282361945|
119.05757732936844| 2| 0| Sud|255458.71474544148|
153.40917758385575| 3| 0| Ouest|267782.27342603524|
107.95302433484777| 3| 0| Ouest|252091.25689685327|
143.50285857656567| 3| 0| Nord| 243798.4680452121|
+------------------+--------+------+-----------+------------------+
only showing top 5 rows

In [0]:
df_train, df_test = sparkDf.randomSplit([0.8, 0.2], seed=91)

string_indexer = StringIndexer(inputCol='orientation', outputCol='orientation'+'Index')
one_hot_encoder = OneHotEncoder(inputCol=string_indexer.getOutputCol(),
                               outputCol='orientation'+'OHE')


In [0]:
label_to_index = StringIndexer(inputCol='price', outputCol='label')

In [0]:
string_model_indexer= string_indexer.fit(df_train)
display(string_model_indexer.transform(df_train))

size,nb_rooms,garden,orientation,price,orientationIndex
1.5270783669052437E-4,3,1,Est,87006.8713280116,2.0
0.0011854990506208196,3,0,Nord,20053.315860567403,3.0
0.0016584871017357727,1,1,Sud,70074.57009988211,0.0
0.002033841565207695,2,1,Sud,90091.42992492826,0.0
0.0024152917479796088,1,0,Ouest,1108.5570828557711,1.0
0.002756015342612273,2,1,Nord,50123.85010231184,3.0
0.002793936561118926,1,1,Ouest,51125.55183487789,1.0
0.0038327291516679907,1,0,Est,-2827.856866850937,2.0
0.004584229515756988,2,1,Sud,90205.81892722787,0.0
0.005062095553228119,3,1,Nord,70227.21968001034,3.0


In [0]:
from pyspark.ml.feature import VectorAssembler
numeric_cols = ["orientationOHE","size","nb_rooms","garden"]
assembler_inputs = numeric_cols
vec_assembler = VectorAssembler(inputCols=assembler_inputs, outputCol='features')


In [0]:
from pyspark.ml.regression import LinearRegression
lr = LinearRegression(featuresCol = 'features', labelCol='label',regParam=0.3)




In [0]:
from pyspark.ml import Pipeline
 
pipeline = Pipeline(stages=[string_indexer, one_hot_encoder, label_to_index, vec_assembler, lr])
pipeline_model = pipeline.fit(df_train)


--------------------------------------------------------------------------- 
 Py4JJavaError Traceback (most recent call last)
 <command-2515566520948288> in <module> 
 2 
 3 pipeline = Pipeline ( stages = [ string_indexer , one_hot_encoder , label_to_index , vec_assembler , lr ] ) 
 ----> 4 pipeline_model = pipeline . fit ( df_train ) 

 /databricks/spark/python/pyspark/ml/base.py in fit (self, dataset, params) 
 159 return self . copy ( params ) . _fit ( dataset ) 
 160 else : 
 --> 161 return self . _fit ( dataset ) 
 162 else : 
 163 raise ValueError("Params must be either a param map or a list/tuple of param maps, "

 /databricks/spark/python/pyspark/ml/pipeline.py in _fit (self, dataset) 
 112 dataset = stage . transform ( dataset ) 
 113 else : # must be an Estimator 
 --> 114 model = stage . fit ( dataset ) 
 115 transformers . append ( model ) 
 116 if i < indexOfLastEstimator : 

 /databricks/spark/python/pyspark/ml/base.py in fit (self, dataset, params) 
 159 return self . copy ( params ) . _fit ( dataset ) 
 160 else : 
 --> 161 return self . _fit ( dataset ) 
 162 else : 
 163 raise ValueError("Params must be either a param map or a list/tuple of param maps, "

 /databricks/spark/python/pyspark/ml/wrapper.py in _fit (self, dataset) 
 333 
 334 def _fit ( self , dataset ) : 
 --> 335 java_model = self . _fit_java ( dataset ) 
 336 model = self . _create_model ( java_model ) 
 337 return self . _copyValues ( model ) 

 /databricks/spark/python/pyspark/ml/wrapper.py in _fit_java (self, dataset) 
 330 """
 331 self . _transfer_params_to_java ( ) 
 --> 332 return self . _java_obj . fit ( dataset . _jdf ) 
 333 
 334 def _fit ( self , dataset ) : 

 /databricks/spark/python/lib/py4j-0.10.9-src.zip/py4j/java_gateway.py in __call__ (self, *args) 
 1302 
 1303 answer = self . gateway_client . send_command ( command ) 
 -> 1304 return_value = get_return_value(
 1305 answer, self.gateway_client, self.target_id, self.name)
 1306 

 /databricks/spark/python/pyspark/sql/utils.py in deco (*a, **kw) 
 108 def deco ( * a , ** kw ) : 
 109 try : 
 --> 110 return f ( * a , ** kw ) 
 111 except py4j . protocol . Py4JJavaError as e : 
 112 converted = convert_exception ( e . java_exception ) 

 /databricks/spark/python/lib/py4j-0.10.9-src.zip/py4j/protocol.py in get_return_value (answer, gateway_client, target_id, name) 
 324 value = OUTPUT_CONVERTER [ type ] ( answer [ 2 : ] , gateway_client ) 
 325 if answer [ 1 ] == REFERENCE_TYPE : 
 --> 326 raise Py4JJavaError(
 327 "An error occurred while calling {0}{1}{2}.\n" . 
 328 format(target_id, ".", name), value)

 Py4JJavaError : An error occurred while calling o2466.fit.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 3 in stage 29.0 failed 1 times, most recent failure: Lost task 3.0 in stage 29.0 (TID 95) (ip-10-172-186-248.us-west-2.compute.internal executor driver): java.lang.OutOfMemoryError: Java heap space

Driver stacktrace:
	at org.apache.spark.scheduler.DAGScheduler.failJobAndIndependentStages(DAGScheduler.scala:2711)
	at org.apache.spark.scheduler.DAGScheduler.$anonfun$abortStage$2(DAGScheduler.scala:2658)
	at org.apache.spark.scheduler.DAGScheduler.$anonfun$abortStage$2$adapted(DAGScheduler.scala:2652)
	at scala.collection.mutable.ResizableArray.foreach(ResizableArray.scala:62)
	at scala.collection.mutable.ResizableArray.foreach$(ResizableArray.scala:55)
	at scala.collection.mutable.ArrayBuffer.foreach(ArrayBuffer.scala:49)
	at org.apache.spark.scheduler.DAGScheduler.abortStage(DAGScheduler.scala:2652)
	at org.apache.spark.scheduler.DAGScheduler.$anonfun$handleTaskSetFailed$1(DAGScheduler.scala:1255)
	at org.apache.spark.scheduler.DAGScheduler.$anonfun$handleTaskSetFailed$1$adapted(DAGScheduler.scala:1255)
	at scala.Option.foreach(Option.scala:407)
	at org.apache.spark.scheduler.DAGScheduler.handleTaskSetFailed(DAGScheduler.scala:1255)
	at org.apache.spark.scheduler.DAGSchedulerEventProcessLoop.doOnReceive(DAGScheduler.scala:2919)
	at org.apache.spa

In [0]:
df_pred = pipeline_model.transform(df_test)
display(df_pred)

--------------------------------------------------------------------------- 
 NameError Traceback (most recent call last)
 <command-1373947216894846> in <module> 
 ----> 1 df_pred = pipeline_model . transform ( df_test ) 
 2 display ( df_pred ) 

 NameError : name 'pipeline_model' is not defined

In [0]:
df_pred.write.csv('predictions.csv')